In [1]:
from collections import defaultdict
import os
import pickle
import random
import requests
import time
import tqdm

from IPython.core.debugger import set_trace
import numpy as np
import pandas as pd
from ranger import Ranger
import torch
import torch.nn as nn
import torch.nn.functional as F 
import torch.utils.data as td
from torch.utils.tensorboard import SummaryWriter

from utils import (EvalDataset, OUNoise, Prioritized_Buffer, get_beta, 
                   preprocess_data, to_np, hit_metric, dcg_metric)

In [2]:
torch.cuda.is_available()

True

In [3]:
data_dir = "data"
rating = "books_3000_new.csv"

params = {
    'batch_size': 512,
    'embedding_dim': 8,
    'hidden_dim': 16,
    'N': 5, # memory size for state_repr
    'ou_noise':False,
    
    'value_lr': 1e-5,
    'value_decay': 1e-4,
    'policy_lr': 1e-5,
    'policy_decay': 1e-6,
    'state_repr_lr': 1e-5,
    'state_repr_decay': 1e-3,
    'log_dir': 'logs/final/',
    'gamma': 0.8,
    'min_value': -10,
    'max_value': 10,
    'soft_tau': 1e-3,
    
    'buffer_size': 1000000
}

## 0. Problem statement

Traditional recommendation task can be treated as sequental desicion making problem.
Recommender (i.e. agent) interacts with users (i.e. environment) to sequentally suggest set of items.
The goal is to maximize clients' satisfaction (i.e. reward).
More specifically:
- State is a vector $a \in R^{3\cdot embedding\_dim}$ computed using the user embedding and the embeddings of `N` latest positive interactions. In the code (replay buffer) state is represented  by `(user, memory)`
- Action is a vector $a \in R^{embedding\_dim}$. To get ranking score we took dot product of
the action and the item embedding (similar to word2vec and other embedding models).
- Reward is taken from user-item matrix (1 if rating > 3, 0 otherwise)

Reinforcement Learning can help recommendation at least in 2 ways.
1. User’s preference on previous items will affect his choice on the next items. 
User tends to give a higher rating if he has consecutively received more satisfied items (and vice versa). 
So, it would be more reasonable to model the recommendation as a sequential decision making process.
2. It is important to use long-term planning in recommendations. For example, after reading the weather forecast, the user is not willing
to read similar news. On the other hand, after watching funny videos or reading memes the user can constanly do the same.

In [4]:
# Movielens (1M) data from the https://github.com/hexiangnan/neural_collaborative_filtering
if not os.path.isdir('./data'):
    os.mkdir('./data')
    
file_path = os.path.join(data_dir, rating)
if os.path.exists(file_path):
    print("Skip loading " + file_path)
else:
    with open(file_path, "wb") as tf:
        print("Load " + file_path)
        r = requests.get("https://raw.githubusercontent.com/hexiangnan/neural_collaborative_filtering/master/Data/" + rating)
        tf.write(r.content)
        
(train_data, train_matrix, test_data, test_matrix, 
 user_num, item_num, appropriate_users) = preprocess_data(data_dir, rating)
print(train_matrix)
print(test_matrix)

Skip loading data/books_3000_new.csv
data:         user   item
0         4     13
1        13     29
2         4     29
4         6     10
5        20     10
...     ...    ...
98603     1  83378
98604    18  83380
98606     1  83383
98607    18  83384
98608    18  83385

[86205 rows x 2 columns]
  (6, 9)	1.0
  (4, 12)	1.0
  (4, 18)	1.0
  (14, 40)	1.0
  (4, 28)	1.0
  (2, 24)	1.0
  (2, 20)	1.0
  (18, 42)	1.0
  (8, 8)	1.0
  (8, 26)	1.0
  (4, 26)	1.0
  (20, 1)	1.0
  (2, 50)	1.0
  (2, 21)	1.0
  (4, 69)	1.0
  (0, 3)	1.0
  (8, 62)	1.0
  (12, 62)	1.0
  (9, 68)	1.0
  (2, 3)	1.0
  (0, 6)	1.0
  (20, 6)	1.0
  (4, 3)	1.0
  (2, 6)	1.0
  (8, 17)	1.0
  :	:
  (14, 83217)	1.0
  (18, 83219)	1.0
  (13, 53704)	1.0
  (19, 83233)	1.0
  (19, 83243)	1.0
  (19, 83246)	1.0
  (19, 83247)	1.0
  (19, 83245)	1.0
  (17, 71743)	1.0
  (19, 83264)	1.0
  (14, 83267)	1.0
  (14, 83268)	1.0
  (16, 82003)	1.0
  (18, 83307)	1.0
  (14, 83330)	1.0
  (14, 83344)	1.0
  (14, 83340)	1.0
  (19, 83348)	1.0
  (14, 83349)	1.0
  (14, 8

## 1. Environment

- **Observation space**. As mentioned before, to get state we need `N` latest positive items (`memory`) and embedding of user. `State_Repr_Module` transform it to the vector of dimensionality `embedding_dim * 3`.

- **Action space**. For every user we sample nonrelated items (the same count as related). All `available_items` which wasn't viewed before form action space.

Given a state we get action embedding, compute dot product between this embedding and embeddings of all items in action space, take 1 top ranked item, compute reward, update `viewed_items` and memory, and store transition in buffer.

In [5]:
class Env():
    def __init__(self, user_item_matrix):
        self.matrix = user_item_matrix
        self.item_count = item_num
        self.memory = np.ones([user_num, params['N']]) * item_num
        # memory is initialized as [item_num] * N for each user
        # it is padding indexes in state_repr and will result in zero embeddings

    def reset(self, user_id):
        self.user_id = user_id
        self.viewed_items = []
        self.related_items = np.argwhere(self.matrix[self.user_id] > 0)[:, 1]
        self.num_rele = len(self.related_items)
        self.nonrelated_items = np.random.choice(
            list(set(range(self.item_count)) - set(self.related_items)), self.num_rele)
        self.available_items = np.zeros(self.num_rele * 2)
        self.available_items[::2] = self.related_items
        self.available_items[1::2] = self.nonrelated_items
        
        return torch.tensor([self.user_id]), torch.tensor(self.memory[[self.user_id], :])
    
    def step(self, action, action_emb=None, buffer=None):
        initial_user = self.user_id
        initial_memory = self.memory[[initial_user], :]
        
        reward = float(to_np(action)[0] in self.related_items)
        self.viewed_items.append(to_np(action)[0])
        if reward:
            if len(action) == 1:
                self.memory[self.user_id] = list(self.memory[self.user_id][1:]) + [action]
            else:
                self.memory[self.user_id] = list(self.memory[self.user_id][1:]) + [action[0]]
                
        if len(self.viewed_items) == len(self.related_items):
            done = 1
        else:
            done = 0
            
        if buffer is not None:
            buffer.push(np.array([initial_user]), np.array(initial_memory), to_np(action_emb)[0], 
                        np.array([reward]), np.array([self.user_id]), self.memory[[self.user_id], :], np.array([reward]))

        return torch.tensor([self.user_id]), torch.tensor(self.memory[[self.user_id], :]), reward, done

## 2. Model

### Overall model

<img src="img/full_model.png" width="500" height="350">

In [6]:
class Actor_DRR(nn.Module):
    def __init__(self, embedding_dim, hidden_dim):
        super().__init__()
    
        self.layers = nn.Sequential(
            nn.Linear(embedding_dim * 3, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, embedding_dim)
        )
        
        self.initialize()

    def initialize(self):
        for layer in self.layers:
            if isinstance(layer, nn.Linear):
                nn.init.kaiming_uniform_(layer.weight)

    def forward(self, state):
        return self.layers(state)
    
    def get_action(self, user, memory, state_repr, 
                   action_emb,
                   items=torch.tensor([i for i in range(item_num)]),
                   return_scores=False
                  ):
        state = state_repr(user, memory)
        items = items.cuda()
        
#         print('itmes', items.shape)
#         print('=======================================')
#         print('state_repr.item_embeddings', state_repr.item_embeddings(items).shape)
#         print('=======================================')
#         print('state_repr.item_embeddings(items).unsqueeze(0)', state_repr.item_embeddings(items).unsqueeze(0).shape)
#         print('=======================================')
#         print('action_emb.T', action_emb.T.shape)
#         print('=======================================')
#         print('action_emb.T.unsqueeze(0)',action_emb.T.unsqueeze(0).shape)
#         print('=======================================')
        
        scores = torch.bmm(state_repr.item_embeddings(items).unsqueeze(0), 
                         action_emb.T.unsqueeze(0)).squeeze(0)
        
#         print('=======================================')
#         print('scores not squeeze', torch.bmm(state_repr.item_embeddings(items).unsqueeze(0), 
#                          action_emb.T.unsqueeze(0)).shape)
#         print('scores', scores.shape)
        
#         raise Exception
        
        if return_scores:
            return scores, torch.gather(items, 0, scores.argmax(0))
        else:
            return torch.gather(items, 0, scores.argmax(0))

In [7]:
class Critic_DRR(nn.Module):
    def __init__(self, state_repr_dim, action_emb_dim, hidden_dim):
        super().__init__()

        self.layers = nn.Sequential(
            nn.Linear(state_repr_dim + action_emb_dim, hidden_dim), 
            nn.ReLU(), 
            nn.Linear(hidden_dim, 1)
        )

        self.initialize()
        
    def initialize(self):
        for layer in self.layers:
            if isinstance(layer, nn.Linear):
                nn.init.kaiming_uniform_(layer.weight)
        
    def forward(self, state, action):
        x = torch.cat([state, action], 1)
        x = self.layers(x)
        return x

### State representation

<img src="img/state_representation.png" width="350" height="250">

In [8]:
class State_Repr_Module(nn.Module):
    def __init__(self, user_num, item_num, embedding_dim, hidden_dim):
        super().__init__()
        self.user_embeddings = nn.Embedding(user_num, embedding_dim)
        self.item_embeddings = nn.Embedding(item_num+1, embedding_dim, padding_idx=int(item_num))
        self.drr_ave = torch.nn.Conv1d(in_channels=params['N'], out_channels=1, kernel_size=1)
        
        self.initialize()
            
    def initialize(self):
        nn.init.normal_(self.user_embeddings.weight, std=0.01)
        nn.init.normal_(self.item_embeddings.weight, std=0.01)
        self.item_embeddings.weight.data[-1].zero_()
        nn.init.uniform_(self.drr_ave.weight)
        self.drr_ave.bias.data.zero_()

    def forward(self, user, memory):
        user_embedding = self.user_embeddings(user.long().cuda())

        item_embeddings = self.item_embeddings(memory.long().cuda())
        drr_ave = self.drr_ave(item_embeddings).squeeze(1)
        
        return torch.cat((user_embedding, user_embedding * drr_ave, drr_ave), 1)

For evaluation we take 1 positive and 99 sampled negatives items per batch, select 10 items with best scores and calculate hit_rate@10 and nDCG@10.
During training we choose user 6039 and track `hit` and `dcg` only for him (for evaluation speed). Final scores was computed on the whole test data.

In [9]:
valid_dataset = EvalDataset(
    np.array(test_data)[np.array(test_data)[:, 0] == 12], 
    item_num, 
    test_matrix)
valid_loader = td.DataLoader(valid_dataset, batch_size=100, shuffle=False)

full_dataset = EvalDataset(np.array(test_data), item_num, test_matrix)
full_loader = td.DataLoader(full_dataset, batch_size=100, shuffle=False)

Resetting dataset
[[12, 62], [12, 47681], [12, 36097], [12, 1284], [12, 66084], [12, 2415], [12, 27955], [12, 44619], [12, 35474], [12, 53252], [12, 67125], [12, 20047], [12, 80973], [12, 841], [12, 55094], [12, 63136], [12, 72788], [12, 35083], [12, 74330], [12, 68637], [12, 70326], [12, 70530], [12, 27234], [12, 5187], [12, 42032], [12, 56762], [12, 4149], [12, 15682], [12, 78341], [12, 22956], [12, 14325], [12, 72670], [12, 64524], [12, 41532], [12, 1034], [12, 62385], [12, 69651], [12, 75648], [12, 20512], [12, 63735], [12, 38149], [12, 11058], [12, 15459], [12, 31698], [12, 51090], [12, 39445], [12, 43570], [12, 2805], [12, 64720], [12, 45096], [12, 28590], [12, 16590], [12, 17302], [12, 27655], [12, 23114], [12, 58297], [12, 81021], [12, 53909], [12, 13142], [12, 35014], [12, 12259], [12, 27971], [12, 18576], [12, 41344], [12, 9527], [12, 43311], [12, 49384], [12, 2200], [12, 42122], [12, 28384], [12, 41035], [12, 72480], [12, 78970], [12, 31059], [12, 22968], [12, 62331], [12, 5

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [10]:
def run_evaluation(net, state_representation, training_env_memory, loader=valid_loader):
    hits = []
    dcgs = []
    test_env = Env(test_matrix)
    test_env.memory = training_env_memory.copy()
    user, memory = test_env.reset(int(to_np(next(iter(valid_loader))['user'])[0]))
    for batch in loader:
        action_emb = net(state_repr(user, memory))
        scores, action = net.get_action(
            batch['user'], 
            torch.tensor(test_env.memory[to_np(batch['user']).astype(int), :]), 
            state_representation, 
            action_emb,
            batch['item'].long(), 
            return_scores=True
        )
        user, memory, reward, done = test_env.step(action)

        _, ind = scores[:, 0].topk(10)
        predictions = torch.take(batch['item'].cuda(), ind).cpu().numpy().tolist()
        actual = batch['item'][0].item()
        hits.append(hit_metric(predictions, actual))
        dcgs.append(dcg_metric(predictions, actual))
        
    return np.mean(hits), np.mean(dcgs)

## 3. Training

In [11]:
torch.manual_seed(2) # cuda의 랜덤성 고정
# np.randomseed: np의 랜덤성 고정

state_repr = State_Repr_Module(user_num, item_num, params['embedding_dim'], params['hidden_dim']).cuda()
policy_net = Actor_DRR(params['embedding_dim'], params['hidden_dim']).cuda()
value_net  = Critic_DRR(params['embedding_dim'] * 3, params['embedding_dim'], params['hidden_dim']).cuda()
replay_buffer = Prioritized_Buffer(params['buffer_size'])

target_value_net  = Critic_DRR(params['embedding_dim'] * 3, params['embedding_dim'], params['hidden_dim']).cuda()
target_policy_net = Actor_DRR(params['embedding_dim'], params['hidden_dim']).cuda()

for target_param, param in zip(target_value_net.parameters(), value_net.parameters()):
    target_param.data.copy_(param.data)

for target_param, param in zip(target_policy_net.parameters(), policy_net.parameters()):
    target_param.data.copy_(param.data)

value_criterion  = nn.MSELoss()
value_optimizer  = Ranger(value_net.parameters(),  lr=params['value_lr'], 
                          weight_decay=params['value_decay'])
policy_optimizer = Ranger(policy_net.parameters(), lr=params['policy_lr'], 
                          weight_decay=params['policy_decay'])
state_repr_optimizer = Ranger(state_repr.parameters(), lr=params['state_repr_lr'], 
                              weight_decay=params['state_repr_decay'])

writer = SummaryWriter(log_dir=params['log_dir'])

Ranger optimizer loaded. 
Gradient Centralization usage = True
GC applied to both conv and fc layers
Ranger optimizer loaded. 
Gradient Centralization usage = True
GC applied to both conv and fc layers
Ranger optimizer loaded. 
Gradient Centralization usage = True
GC applied to both conv and fc layers


In [12]:
def ddpg_update(training_env, 
                step=0,
                batch_size=params['batch_size'], 
                gamma=params['gamma'],
                min_value=params['min_value'],
                max_value=params['max_value'],
                soft_tau=params['soft_tau'],
               ):
    beta = get_beta(step) # DDPG에 PER parameter
    user, memory, action, reward, next_user, next_memory, done = replay_buffer.sample(batch_size, beta)
    user        = torch.FloatTensor(user).cuda()
    memory      = torch.FloatTensor(memory).cuda()
    action      = torch.FloatTensor(action).cuda()
    reward      = torch.FloatTensor(reward).cuda()
    next_user   = torch.FloatTensor(next_user).cuda()
    next_memory = torch.FloatTensor(next_memory).cuda()
    done = torch.FloatTensor(done).cuda()
    
    state       = state_repr(user, memory)
    policy_loss = value_net(state, policy_net(state))
    policy_loss = -policy_loss.mean()
    
    next_state     = state_repr(next_user, next_memory)
    next_action    = target_policy_net(next_state)
    target_value   = target_value_net(next_state, next_action.detach())
    expected_value = reward + (1.0 - done) * gamma * target_value
    expected_value = torch.clamp(expected_value, min_value, max_value) # 학습의 안정성을 위해서

    value = value_net(state, action)
    value_loss = value_criterion(value, expected_value.detach())
    
    state_repr_optimizer.zero_grad() # Zero_grad() => backward(미분) => step() (업데이트))
    policy_optimizer.zero_grad()
    policy_loss.backward(retain_graph=True)
    policy_optimizer.step()

    value_optimizer.zero_grad()
    value_loss.backward(retain_graph=True)
    value_optimizer.step()
    state_repr_optimizer.step()

    for target_param, param in zip(target_value_net.parameters(), value_net.parameters()):
                target_param.data.copy_(
                    target_param.data * (1.0 - soft_tau) + param.data * soft_tau
                )

    for target_param, param in zip(target_policy_net.parameters(), policy_net.parameters()):
            target_param.data.copy_(
                target_param.data * (1.0 - soft_tau) + param.data * soft_tau
            )

    writer.add_histogram('value', value, step)
    writer.add_histogram('target_value', target_value, step)
    writer.add_histogram('expected_value', expected_value, step)
    writer.add_histogram('policy_loss', policy_loss, step)

In [13]:
np.random.seed(16)
train_env = Env(train_matrix)
hits, dcgs = [], []
hits_all, dcgs_all = [], []
step, best_step = 0, 0
step, best_step, best_step_all = 0, 0, 0
users = np.random.permutation(appropriate_users)
ou_noise = OUNoise(params['embedding_dim'], decay_period=10)

In [31]:
'''
np.random.seed(16)
train_env = Env(train_matrix)
hits, dcgs = [], []
hits_all, dcgs_all = [], []
step, best_step = 0, 0
step, best_step, best_step_all = 0, 0, 0
users = np.random.permutation(appropriate_users)
ou_noise = OUNoise(params['embedding_dim'], decay_period=10)
'''

for u in tqdm.tqdm(users):
    user, memory = train_env.reset(u)
    if params['ou_noise']:
        ou_noise.reset()
    for t in range(int(train_matrix[u].sum())):
        action_emb = policy_net(state_repr(user, memory))
        if params['ou_noise']:
            action_emb = ou_noise.get_action(action_emb.detach().cpu().numpy()[0], t)
        action = policy_net.get_action(
            user, 
            torch.tensor(train_env.memory[to_np(user).astype(int), :]), 
            state_repr, 
            action_emb,
            torch.tensor(
                [item for item in train_env.available_items 
                if item not in train_env.viewed_items]
            ).long()
        )
        user, memory, reward, done = train_env.step(
            action, 
            action_emb,
            buffer=replay_buffer
        ) # user, memory 받을 필요 없어보임

        if len(replay_buffer) > params['batch_size']:
            ddpg_update(train_env, step=step)

        if step % 100 == 0 and step > 0:
            hit, dcg = run_evaluation(policy_net, state_repr, train_env.memory)
            writer.add_scalar('hit', hit, step)
            writer.add_scalar('dcg', dcg, step)
            hits.append(hit)
            dcgs.append(dcg)
            if np.mean(np.array([hit, dcg]) - np.array([hits[best_step], dcgs[best_step]])) > 0:
                best_step = step // 100
                torch.save(policy_net.state_dict(), params['log_dir'] + 'policy_net.pth')
                torch.save(value_net.state_dict(), params['log_dir'] + 'value_net.pth')
                torch.save(state_repr.state_dict(), params['log_dir'] + 'state_repr.pth')
        if step % 10000 == 0 and step > 0:
            hit, dcg = run_evaluation(policy_net, state_repr, train_env.memory, full_loader)
            writer.add_scalar('hit_all', hit, step)
            writer.add_scalar('dcg_all', dcg, step)
            hits_all.append(hit)
            dcgs_all.append(dcg)
            if np.mean(np.array([hit, dcg]) - np.array([hits_all[best_step_all], dcgs_all[best_step_all]])) > 0:
                best_step_all = step // 10000
                torch.save(policy_net.state_dict(), params['log_dir'] + 'best_policy_net.pth')
                torch.save(value_net.state_dict(), params['log_dir'] + 'best_value_net.pth')
                torch.save(state_repr.state_dict(), params['log_dir'] + 'best_state_repr.pth')
        step += 1

  0%|          | 0/22 [00:00<?, ?it/s]/home/aiffel0042/anaconda3/envs/aiffel/lib/python3.7/site-packages/ranger/ranger.py:138: UserWarning: This overload of addcmul_ is deprecated:
	addcmul_(Number value, Tensor tensor1, Tensor tensor2)
Consider using one of the following signatures instead:
	addcmul_(Tensor tensor1, Tensor tensor2, *, Number value) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)
100%|██████████| 22/22 [10:54:13<00:00, 1784.27s/it]  


In [32]:
torch.save(policy_net.state_dict(), params['log_dir'] + 'policy_net_final.pth')
torch.save(value_net.state_dict(), params['log_dir'] + 'value_net_final.pth')
torch.save(state_repr.state_dict(), params['log_dir'] + 'state_repr_final.pth')

In [14]:
# we need memory for validation, so it's better to save it and not wait next time 
with open('logs/memory.pickle', 'wb') as f:
    pickle.dump(train_env.memory, f)
    
with open('logs/memory.pickle', 'rb') as f:
    memory = pickle.load(f)

## 4. Results

Weights and logs are stored in [this folder](https://drive.google.com/drive/folders/1oz6w4HBEVspvReBQ55zeIE4qw5OPD84U?usp=sharing)

In [38]:
no_ou_state_repr = State_Repr_Module(user_num, item_num, params['embedding_dim'], params['hidden_dim'])
no_ou_policy_net = Actor_DRR(params['embedding_dim'], params['hidden_dim'])
no_ou_state_repr.load_state_dict(torch.load('logs/no_ou/' + 'best_state_repr.pth'))
no_ou_policy_net.load_state_dict(torch.load('logs/no_ou/' + 'best_policy_net.pth'))
    
hit, dcg = run_evaluation(no_ou_policy_net, no_ou_state_repr, memory, full_loader)
print('hit rate: ', hit, 'dcg: ', dcg)

RuntimeError: Error(s) in loading state_dict for State_Repr_Module:
	size mismatch for user_embeddings.weight: copying a param with shape torch.Size([6040, 8]) from checkpoint, the shape in current model is torch.Size([83386, 8]).
	size mismatch for item_embeddings.weight: copying a param with shape torch.Size([3707, 8]) from checkpoint, the shape in current model is torch.Size([23, 8]).

In [21]:
_state_repr = State_Repr_Module(user_num, item_num, params['embedding_dim'], params['hidden_dim']).cuda()
_policy_net = Actor_DRR(params['embedding_dim'], params['hidden_dim']).cuda()
_state_repr.load_state_dict(torch.load(params['log_dir'] + 'state_repr_final.pth'))
_policy_net.load_state_dict(torch.load(params['log_dir'] + 'policy_net_final.pth'))

hit, dcg = run_evaluation(_policy_net, _state_repr, memory, full_loader)
print('hit rate: ', hit, 'dcg: ', dcg)

hit rate:  0.035148773273012 dcg:  0.016281878010435697


### Example of trained agents behaviour

Let's choose random user

In [18]:
random_user = np.random.randint(user_num)
print(random_user)

19


In [ ]:
movies = pd.read_csv('data/movies.dat', sep='::', header=None, engine='python', names=['id', 'name', 'genre'])
# in the code numeration starts with 0
movies[movies['id'].isin(np.argwhere(test_matrix[random_user] > 0)[:, 1] + 1)]

For example we can recommend "Nixon" and "Love Serenade" and see next 3 predictions.

In [ ]:
predictions = []

for model, state_representation in zip([ou_policy_net, no_ou_policy_net], [ou_state_repr, no_ou_state_repr]):
    example_env = Env(test_matrix)
    user, memory = example_env.reset(random_user)

    user, memory, reward, _ = example_env.step(torch.tensor([13]))
    user, memory, reward, _ = example_env.step(torch.tensor([1584]))
    preds = []
    for _ in range(3):
        action_emb = model(state_representation(user, memory))
        action = model.get_action(
            user, 
            torch.tensor(example_env.memory[to_np(user).astype(int), :]), 
            state_representation, 
            action_emb,
            torch.tensor(
                [item for item in example_env.available_items 
                if item not in example_env.viewed_items]
            ).long()
        )
        user, memory, reward, _ = example_env.step(action)
        preds.append(action)

    predictions.append(preds)

print(predictions[0])
print(predictions[1])

Model trained with OU noise recommended related `Comedy` and `Documentary` after that switch recommendations to nonrelated `Crime|Film-Noir|Thriller`.
Model trained without OU noise recommended `Comedy|Drama`, `Children's|Comedy`, `Drama` (two of them are related).

Both models seems to be reasonable.

### Training process logs

<img src=img/learning_curve.png>

Logs are consistent with expectations. Adding noise increase metrics (std=0.4 performs the best, after 0.6 model starts to degrade).